# Project Title
### Data Engineering Capstone Project

#### Project Summary
This project  prepares the immigration, demographic, airpot and temperature data so that the resulting tables are ready to be loaded into a OLAP tabular model such as Microsoft SQL Server Analysis Services or Microsoft PowerBI. The immigration table will be the fact table with records of individuals immigrating to the United sStates. While the remaining tables will be dimension tables that help expand the other atttributes of each record in the fact table. These dimension tables include Demographics which will include data regarding the demographics of US Cities, Temperatures will hold data pertaining to the temperatures of OS Cities, and Airports will contain US Airport information.


The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

1. **U.S. City Demographic Data:** comes from [OpenSoft](https://public.opendatasoft.com) and includes data by city, state, age, population, veteran status and race.
  
2. **I94 Immigration Data:** comes from the [US National Tourism and Trade Office](https://travel.trade.gov/research/reports/i94/historical/2016.html) and includes details on incoming immigrants and their ports of entry

3. **Airport Code Table:** comes from [datahub.io](https://datahub.io/core/airport-codes#data) and includes airport codes and corresponding cities.

4. **World Temperature Data:** comes from [kaggle](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data) and includes data on temperature changes in the U.S. since 1850."
   

## 1.1 Configurations and Set Up

In [1]:
# All necessary imports and installs
import pandas as pd
from pyspark.sql import SparkSession, SQLContext
import configparser
import re
from pyspark.sql.functions import *
from collections import  defaultdict
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [2]:
# From Harutaka K. on Student Hub Thread. https://hub.udacity.com/rooms/community:nd027:843753-project-577/community:thread-u22525051-879375?contextType=room
def parse_countries(text):
  contries_text = re.search(r'I94CIT.+?;', text, re.DOTALL).group(0)
  matches = re.findall(r'(.+)=(.+)', contries_text)
  matches.sort(key=lambda m: int(m[0]))
  countries = {}

  for (cid, cname) in matches:
    countries[int(cid.strip())] = cname.strip()[1:-1]
  return countries

with open('I94_SAS_Labels_Descriptions.SAS', 'r') as f:
  text = f.read()

countries = parse_countries(text)
list(countries.items())[:5]

[(0, 'INVALID: STATELESS'),
 (54, 'No Country Code (54)'),
 (100, 'No Country Code (100)'),
 (101, 'ALBANIA'),
 (102, 'ANDORRA')]

## 1.2 Import and Explore Data

In [3]:
# Read airport code data:
airport_codes_df = pd.read_csv('data/airport-codes.csv', header=0, sep=',')

# Read airport code data:
demographic_df = pd.read_csv('data/us-cities-demographics.csv', header=0, sep=';')

# Read Global country codes data:
country_codes_df = pd.DataFrame(list(countries.items()), columns =['Country_Code', 'Country'])

# Read City Temperature Data
temperature_df =  pd.read_csv('../../data2/GlobalLandTemperaturesByCity.csv', header=0, sep=',')

#Read Immigration Data
fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
immigration_df = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")

# Read in the data here
#Postal Codes: https://fam.state.gov/fam/09FAM/09FAM010205.html
postal_codes_df = pd.read_csv('data/PostCodes.csv', header=0, sep=',')


In [4]:
airport_codes_df.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [5]:
demographic_df.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [6]:
country_codes_df.head()

,Country_Code,Country
0,0,INVALID: STATELESS
1,54,No Country Code (54)
2,100,No Country Code (100)
3,101,ALBANIA
4,102,ANDORRA


In [7]:
temperature_df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [8]:
immigration_df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


In [9]:
#Create a Dictionary from the Postal Codes to look up later
code_dict = defaultdict(list)

for code,city,state in zip(postal_codes_df.Code, postal_codes_df.City, postal_codes_df.State_Code):
    code_dict[code] = [city, state]

postal_codes_df.head()


,Code,City,State_Code,Region,Country
0,ABE,Aberdeen,WA,NaN,NaN
1,ABG,Alburg,VT,NaN,NaN
2,ABQ,Albuquerque,NM,NaN,NaN
3,ABS,AlburgSprings,VT,NaN,NaN
4,ADT,AmistadDam,TX,NaN,NaN


## 1.3: Start Spark Session

In [10]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
    config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
    .enableHiveSupport().getOrCreate()

#Build SQL context object
sqlContext = SQLContext(spark)

### 1.4 Import Data into Spark

In [11]:
demographics_spark_df=spark.read.format("csv").option("header", "true").option("delimiter", ";").load("us-cities-demographics.csv")
airport_spark_df=spark.read.format("csv").option("header", "true").load("airport-codes_csv.csv")
temperature_spark_df=spark.read.format("csv").option("header", "true").load("../../data2/GlobalLandTemperaturesByCity.csv")
country_code_spark_df = spark.createDataFrame(country_codes_df)
immigration_spark_df=spark.read.format('com.github.saurfang.sas.spark').load("../../data/18-83510-I94-Data-2016/i94_jun16_sub.sas7bdat")

### Step 2: Explore and Assess the Data

<!--
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data
-->

#### Temperature Data by City, Date

In [12]:
## Temperature Data

Temperature_Clean=temperature_spark_df\
    .filter(temperature_spark_df["country"]=="United States")\
    .filter(year(temperature_spark_df["dt"])==2013)\
    .withColumn("year",year(temperature_spark_df["dt"]))\
    .withColumn("month",month(temperature_spark_df["dt"]))\
    .withColumn("avg_temp_fahrenheit",temperature_spark_df["AverageTemperature"]*9/5+32)\

Temperatures=Temperature_Clean.select("year","month","Country", "City", round(col("AverageTemperature"),1).alias("avg_temp_celcius"),round(col("avg_temp_fahrenheit"),1).alias("avg_temp_fahrenheit")).dropDuplicates()


In [13]:
#print(Temperature_df.columns)
print(Temperatures.columns)
#print(Temperature_Clean.show(1))

# Might need City , State Key

['year', 'month', 'Country', 'City', 'avg_temp_celcius', 'avg_temp_fahrenheit']


#### Immigration Data by City, State, Month, Year

In [14]:

Immigration_Clean=immigration_spark_df\
    .filter(immigration_spark_df.i94addr.isNotNull())\
    .filter(immigration_spark_df.i94res.isNotNull())\
    .withColumn("i94yr",col("i94yr").cast("integer"))\
    .withColumn("i94mon",col("i94mon").cast("integer")\
    )

# Select specifc columns
Immigration=Immigration_Clean.select(\
        "cicid",col("i94yr").alias("year")\
        ,col("i94mon").alias("month")\
        ,"i94port"\
        ,col("i94addr").alias("state")\
        , "biryear"\
        , "gender"\
        , "airline"\
        , "visatype"\
        )
        #,'visapost', 'entdepa', 'entdepd', 'entdepu', 'matflag', 'biryear', 'dtaddto', 'gender', 'insnum', 'airline', 'admnum', 'fltno', 'visatype'\



In [15]:
#print(Immigration_Clean.columns)
#print(Immigration_Clean.head(1))
#print("--------------")
print(Immigration.show(1))
print(Immigration.columns)
# Might need City , State Key

+-----+----+-----+-------+-----+-------+------+-------+--------+
|cicid|year|month|i94port|state|biryear|gender|airline|visatype|
+-----+----+-----+-------+-----+-------+------+-------+--------+
| 41.0|2016|    6|    SFR|   CA| 1994.0|     F|     KE|      F1|
+-----+----+-----+-------+-----+-------+------+-------+--------+
only showing top 1 row

None
['cicid', 'year', 'month', 'i94port', 'state', 'biryear', 'gender', 'airline', 'visatype']


#### Demographic Data by City, State

In [16]:


#Calculate percentages of each numeric column and create new columns.
Demographics_Clean=demographics_spark_df\
    .withColumn("Median_Age",col("Median Age").cast("float"))\
    .withColumn("%_Male",demographics_spark_df["Male Population"]/demographics_spark_df["Total Population"]*100)\
    .withColumn("%_Female",demographics_spark_df["Female Population"]/demographics_spark_df["Total Population"]*100)\
    .withColumn("%_Veterans",demographics_spark_df["Number of Veterans"]/demographics_spark_df["Total Population"]*100)\
    .withColumn("%_Foreign_Born",demographics_spark_df["Foreign-born"]/demographics_spark_df["Total Population"]*100)\
    .withColumn("%_Race",demographics_spark_df["Count"]/demographics_spark_df["Total Population"]*100)\
    .orderBy("State")

#    .withColumn("pct_race",demog["Count"]/demog["Total Population"]*100)\

#Select desiredd columns and new calculated percentages 
Demographics_Clean=Demographics_Clean.select("City","State","State Code","Median_Age", "%_Male", "%_Female", "%_Veterans", "%_Foreign_Born", "Race", "%_Race")


#pivot the Race column
Demographics=Demographics_Clean.groupBy("City","State", "State Code","Median_Age", "%_Male", "%_Female", "%_Veterans", "%_Foreign_Born").pivot("Race").avg("%_Race")
Demographics=Demographics.select("City","State",col("State Code").alias("State_Code"),"Median_Age", "%_Male", "%_Female", "%_Veterans", "%_Foreign_Born"\
                                       ,col("American Indian and Alaska Native").alias("Native_American")\
                                       , "Asian"\
                                       , col("Black or African-American").alias("Black_or_African-American")\
                                       , col("Hispanic or Latino").alias("Hispanic_or_Latino")\
                                       , "White"\
                                      )



In [17]:
#print(Demographics.columns)
#print(Demographics_Clean.columns)
#print(demographics_spark_df.head(1))
print(Demographics.show(2))
Demographics.columns

+----------+-------+----------+----------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+-------------------------+------------------+------------------+
|      City|  State|State_Code|Median_Age|           %_Male|         %_Female|       %_Veterans|    %_Foreign_Born|   Native_American|            Asian|Black_or_African-American|Hispanic_or_Latino|             White|
+----------+-------+----------+----------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+-------------------------+------------------+------------------+
|    Hoover|Alabama|        AL|      38.5| 44.8378693761124|55.16213062388759| 5.68017067622202| 9.699548556677943|              null|5.609448484777048|       21.441789742924833| 4.042951944270913| 72.92518770848314|
|Montgomery|Alabama|        AL|      35.4|47.15284217243477|52.84715782756524|7.455654931052018|4.6548612565184015|0.636634660444896

['City',
 'State',
 'State_Code',
 'Median_Age',
 '%_Male',
 '%_Female',
 '%_Veterans',
 '%_Foreign_Born',
 'Native_American',
 'Asian',
 'Black_or_African-American',
 'Hispanic_or_Latino',
 'White']

### US Airports by City, State


In [18]:
#Filter airport data for 'small_airport' in the U.S. and use substring to show state
Airports_Clean=airport_spark_df\
    .filter(airport_spark_df["iso_country"]=="US")\
    .withColumn("iso_region",substring(airport_spark_df["iso_region"],4,2))\
    .withColumn("elevation_ft",col("elevation_ft").cast("float"))

#Select relevant columns and drop duplicates
Airports=Airports_Clean.select(col("iso_country").alias("Country"),col("iso_region").alias("State_Code"),col("municipality").alias("City"),"elevation_ft", "type", "name", "ident")

In [19]:
#print(Demographics.columns)
#print(Airports_Clean.columns)
print(Airports.columns)
#print(Airports.head(5))
print(Airports.show(5))


['Country', 'State_Code', 'City', 'elevation_ft', 'type', 'name', 'ident']
+-------+----------+------------+------------+-------------+--------------------+-----+
|Country|State_Code|        City|elevation_ft|         type|                name|ident|
+-------+----------+------------+------------+-------------+--------------------+-----+
|     US|        PA|    Bensalem|        11.0|     heliport|   Total Rf Heliport|  00A|
|     US|        KS|       Leoti|      3435.0|small_airport|Aero B Ranch Airport| 00AA|
|     US|        AK|Anchor Point|       450.0|small_airport|        Lowell Field| 00AK|
|     US|        AL|     Harvest|       820.0|small_airport|        Epps Airpark| 00AL|
|     US|        AR|     Newport|       237.0|       closed|Newport Hospital ...| 00AR|
+-------+----------+------------+------------+-------------+--------------------+-----+
only showing top 5 rows

None


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

The __Immigration__ table will be the fact table with records of individuals immigrating to the United sStates. While the remaining tables will be dimension tables that help expand the other atttributes of each record in the fact table. These dimension tables include __Demographics__ which will include data regarding the demographics of US Cities, __Temperatures__ will hold data pertaining to the temperatures of OS Cities, and __Airports__ will contain US Airport information. The key to relate data from the fact table to the dimension tables will be based on the State and City fields which are present on all tables.



### Star Schema
#### Dimension Tables
- __Airport Table__
    - Columns: 'Country', 'State_Code', 'City', 'elevation_ft', 'type', 'name', 'ident'
    
- __Demographic Table__
    - Columns: 'City',  'State', 'State_Code', 'Median_Age', '%_Male', '%_Female', '%_Veterans', '%_Foreign_Born', 'Native_American', 'Asian', 'Black_or_African-American', 'Hispanic_or_Latino', 'White'
    
- __Temperature Table__
    - Columns: 'year', 'month', 'Country', 'City', 'avg_temp_celcius', 'avg_temp_fahrenheit'
    
#### Fact Table:
- __Immigration Table__
    - Columns: 'cicid', 'year', 'month', 'i94port', 'state', 'biryear', 'gender', 'airline', 'visatype'
    



#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

1. Stage and Read the data into Spark Data frames
2. Implement all required data cleaning and transformation steps
3. Merge and/or Join data from different data sources to fill in any data ga  
4. Save dataframes into parquet files
5. Make files accessible to read for customer use

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [20]:
# Create tables in spark session
Immigration.createOrReplaceTempView("Immigration")
Demographics.createOrReplaceTempView("Demographic")
Airports.createOrReplaceTempView("Airport")
Temperatures.createOrReplaceTempView("Temperature")


In [21]:
# Write fact table to parquet
output_data = "/parquet"
start_time = "2019_09_26"

# Write immigration_table to parquet file:
immigration_table_path = output_data + "/immigration/immigration_table.parquet" + "_" + start_time
print(f"OUTPUT: {immigration_table_path}")
Immigration.write.mode("overwrite").parquet(immigration_table_path)
print("Writing DONE.")

# Write demographic to parquet file:
demographic_table_path = output_data + "/demographic/demographic_table.parquet" + "_" + start_time
print(f"OUTPUT: {demographic_table_path}")
Demographics.write.mode("overwrite").parquet(demographic_table_path)
print("Writing DONE.")

# Write airport to parquet file:
airport_table_path = output_data + "/airport/airport_table.parquet" + "_" + start_time
print(f"OUTPUT: {airport_table_path}")
Airports.write.mode("overwrite").parquet(airport_table_path)
print("Writing DONE.")

# Write Temperatures to parquet file:
temperture_table_path = output_data + "/temperture/temperture_table.parquet" + "_" + start_time
print(f"OUTPUT: {temperture_table_path}")
Temperatures.write.mode("overwrite").parquet(temperture_table_path)
print("Writing DONE.")




OUTPUT: /parquet/immigration/immigration_table.parquet_2019_09_26
Writing DONE.
OUTPUT: /parquet/demographic/demographic_table.parquet_2019_09_26
Writing DONE.
OUTPUT: /parquet/airport/airport_table.parquet_2019_09_26
Writing DONE.
OUTPUT: /parquet/temperture/temperture_table.parquet_2019_09_26
Writing DONE.


In [22]:
# Read parquet file back to Spark:

Immigration = spark.read.parquet(immigration_table_path)
#Demographics = spark.read.parquet(demographic_table_path)
#Airports= spark.read.parquet(airport_table_path)
#Temperatures = spark.read.parquet(temperture_table_path)


# create a function to be applied in in a spark data frame
def get_city(x):
    if len(code_dict[x]) == 0:
        value = None
    else:
        value = code_dict[x][0]
    return value

city_udf_dict = udf(lambda z: get_city(z), StringType())

#Add City name based on port name from Immigration dataframe
Immigration=Immigration.select("cicid"\
    , "year"\
    , "month"\
    ,"i94port"\
    , city_udf_dict('i94port').alias('City')\
    , "state"\
    , "biryear"\
    , "gender"\
    , "airline"\
    , "visatype"\
    )


#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [23]:
# Get a Row COunt for each table. ALl tables should have record counts > 0
print("Immigration Table row count: ", Immigration.count())
print("Demographics Table row count: ", Demographics.count())
print("Temperatures Table row count: ", Temperatures.count())
print("Airports Table row count: ", Airports.count())

Immigration Table row count:  3388925
Demographics Table row count:  596
Temperatures Table row count:  2313
Airports Table row count:  22757


In [24]:
# CHeck for null values
#Cicid requires there be no null values. Any null records for cicid will have to be removed or rechecked.
Immigration.select(isnull('cicid').alias('cicid')\
                    , isnull('year').alias('year')\
                    , isnull('month').alias('month')\
                    , isnull('i94port').alias('i94port')\
                    , isnull('City').alias('City')\
                    , isnull('state').alias('state')\
                   , isnull('biryear').alias('biryear')\
                   , isnull('gender').alias('gender')\
                   , isnull('airline').alias('airline')\
                   , isnull('visatype').alias('visatype')\
                  ).dropDuplicates().show()

+-----+-----+-----+-------+-----+-----+-------+------+-------+--------+
|cicid| year|month|i94port| City|state|biryear|gender|airline|visatype|
+-----+-----+-----+-------+-----+-----+-------+------+-------+--------+
|false|false|false|  false|false|false|  false|  true|   true|   false|
|false|false|false|  false| true|false|  false| false|  false|   false|
|false|false|false|  false|false|false|  false| false|  false|   false|
|false|false|false|  false| true|false|   true| false|   true|   false|
|false|false|false|  false|false|false|  false|  true|  false|   false|
|false|false|false|  false|false|false|   true| false|   true|   false|
|false|false|false|  false| true|false|  false| false|   true|   false|
|false|false|false|  false| true|false|  false|  true|   true|   false|
|false|false|false|  false|false|false|   true| false|  false|   false|
|false|false|false|  false| true|false|   true| false|  false|   false|
|false|false|false|  false|false|false|  false| false|   true|  

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

* Data Dictionary has been added to Data_Dictionary.md


#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

#### Tool and Technology Choices:

For this project, Apache Spark was utilized to read, transform, and store data. The i94 immigration data contained over 3 million recordds of migrant data into the United States.
Utilizing distrbuted technologies for data manipulation like Spark is ideal for working with large data sets.

Python was utilized to interact with spark for easier developer experience. And Python's rich community for module development provides many easy options for data manipulations such as pandas.


#### Data Refresh Schedule

A monthly data refresh is recommended for this data model. Many of the data sources such as Immigration and Temperature have the granularity to break down the data by Month and Year of each record.
These data sources can be exected to be update from the source on a monthly cadence and the pipeline should match to guarantee frequent data updates.

#### Approaching New Data problems

- If the data was increased by 100x, I would implement the following items:
    - Build out specifc key columns across the fact and dimensional tables so that better indexing can be created, making querying the data more efficient.
    - Utilize Hadoop to create a distributed computing data architecture for faster processing of big data.
    
- To update on a daily basis I would implement the following options:
    - Create an Data ETL Orchestration environment that can schedule the data pipeline everyday before 7AM. An example product would be Apache Airflow which is a popular orchestration tool used across the data industry
    - The need for daily data would also require a more scalable data solution. Utilizing AWS S3 buckets to stage and save these data files would be a reliable solution
    
- To make the data accessible to over 100+ people I would implent the following:
    - Create a customer facing front end environment that could allow customers to read from these tables. Viable options include creating a data virtualization layer such as Cisco Data VIrtualization, and allow customers to subscribe to access to certain schemas.
    - Similarly, utlizing OLAP cube services such as Azure Analysis Services is a great option, with public cloud resources able to expand resources as needed when load traffic increases.
    - Another option is to create an API layer which customers can utilize to access the data via code, which they could then ingest for their own data needs
    